In [1]:
%cd ..

/app


In [1]:
import gymnasium as gym

import numpy as np
from tqdm import tqdm

from tensoraerospace.envs import LinearLongitudinalUAV
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

In [2]:
dt = 0.01  # Дискретизация
tp = generate_time_period(tn=20, dt=dt) # Временной периуд
tps = convert_tp_to_sec_tp(tp, dt=dt)
number_time_steps = len(tp) # Количество временных шагов
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=True), [1, -1]) # Заданный сигнал

In [3]:
env = gym.make('LinearLongitudinalUAV-v0',
               number_time_steps=number_time_steps, 
               initial_state=[[0],[0],[0],[0]],
               output_space=None,
               reference_signal = reference_signals)
env.reset()

/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/Users/asmazaev/Projects/TensorAeroSpace/.venv/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(


AttributeError: 'LongitudinalUAV' object has no attribute 'selected_state_index'

In [5]:
observation, reward, done, info = env.step(np.array([[1]]))

In [6]:
env.model.store_input

array([[1., 0., 0., ..., 0., 0., 0.]])

In [7]:
env.reference_signal[0][1]

0.0

In [8]:
reward

array([6.56273773e-05])